# Oakland 311 calls, part II 

## Import modules
Again, at this point we will import the panda and altair modules at the start of the notebook. This time, we won't need to add requirements because we already have access to the data.

In [1]:
import pandas as pd
import altair as alt

/Users/gracelynnewest/.pyenv/versions/3.8.5/envs/lecture_06-3.8.5/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Import data
We did a lot of work last week cleaning up the Berkeley 311 calls. We don't need to redo that work since we exported a clean version called `oakland_311_clean.csv`. 

Remember that a `csv` file is just a plain-text file. That means that the file, just as it is, cannot retain the **dtype** of a column.

So this time when we import the data, we'll want to make sure that we set up the dtypes we do know and parse `datetime` dtypes correctly.

I also want to set **REQUESTID** to an `object` dtype instead of an `int` dtype. Why would I want to do this? You can't operate on **REQUESTID** like it's a number. You aren't going to add up the REQUESTIDs. So it's better to import that column as an `object`.

In [2]:
oakland_311 = pd.read_csv('oakland_311_clean.csv', 
    dtype={
        'REQUESTID': object,
    },
    parse_dates=['DATETIMEINIT', 'DATETIMECLOSED', 'Close_Time']
)

In [3]:
oakland_311.head()

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT,PROBADDRESS,City,State,Close_Time
0,1167791,2021-12-01 08:29:53,Email,Inspections - Sewer Lateral,SEWERS,NaN,OPEN,NaN,NaT,NaN,NaN,NaN,NaN,2122 108TH AV,Oakland,CA,NaN
1,1167997,2021-12-01 14:54:36,Voicemail,Homeless Encampment,HE_CLEAN,NaN,OPEN,NaN,NaT,NaN,NaN,NaN,NaN,OAK GLEN PARK,Oakland,CA,NaN
2,1148022,2021-09-16 21:37:36,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.84456257751645, -122.26704955148797)",OPEN,NaN,NaT,6051460.077,2134803.248,CCD1,11X,5911 MACCALL ST,Oakland,CA,NaN
3,1148574,2021-09-20 13:30:43,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.841942007577984, -122.27307401524136)",CLOSED,NaN,2021-12-01 14:06:40,6049702.742,2133882.102,CCD1,10Y,878 57TH ST,Oakland,CA,72 days 00:35:57
4,1149273,2021-09-22 13:20:59,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.8424970155393, -122.2694559662867)",OPEN,NaN,NaT,6050751.127,2134064.386,CCD1,11X,5714 MARTIN LUTHER KING JR WAY,Oakland,CA,NaN


In [4]:
oakland_311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 862931 entries, 0 to 862930
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   REQUESTID        862931 non-null  object        
 1   DATETIMEINIT     862931 non-null  datetime64[ns]
 2   SOURCE           862931 non-null  object        
 3   DESCRIPTION      862931 non-null  object        
 4   REQCATEGORY      862849 non-null  object        
 5   REQADDRESS       846829 non-null  object        
 6   STATUS           862929 non-null  object        
 7   REFERREDTO       71107 non-null   object        
 8   DATETIMECLOSED   622089 non-null  datetime64[ns]
 9   SRX              846829 non-null  float64       
 10  SRY              846829 non-null  float64       
 11  COUNCILDISTRICT  819703 non-null  object        
 12  BEAT             822301 non-null  object        
 13  PROBADDRESS      862452 non-null  object        
 14  City             862

The **Close_Time** column didn't get typed as `timedelta`. It doesn't look like it's possible to do so with `pd.read_csv()`. So we'll just set it this way:

In [5]:
oakland_311['Close_Time'] = pd.to_timedelta(oakland_311['Close_Time']) 

In [6]:
oakland_311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 862931 entries, 0 to 862930
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype          
---  ------           --------------   -----          
 0   REQUESTID        862931 non-null  object         
 1   DATETIMEINIT     862931 non-null  datetime64[ns] 
 2   SOURCE           862931 non-null  object         
 3   DESCRIPTION      862931 non-null  object         
 4   REQCATEGORY      862849 non-null  object         
 5   REQADDRESS       846829 non-null  object         
 6   STATUS           862929 non-null  object         
 7   REFERREDTO       71107 non-null   object         
 8   DATETIMECLOSED   622089 non-null  datetime64[ns] 
 9   SRX              846829 non-null  float64        
 10  SRY              846829 non-null  float64        
 11  COUNCILDISTRICT  819703 non-null  object         
 12  BEAT             822301 non-null  object         
 13  PROBADDRESS      862452 non-null  object         
 14  City

Now that it has been converted to 'timedelta' we can properly sort the data

In [7]:
oakland_311

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT,PROBADDRESS,City,State,Close_Time
0,1167791,2021-12-01 08:29:53,Email,Inspections - Sewer Lateral,SEWERS,NaN,OPEN,NaN,NaT,NaN,NaN,NaN,NaN,2122 108TH AV,Oakland,CA,NaT
1,1167997,2021-12-01 14:54:36,Voicemail,Homeless Encampment,HE_CLEAN,NaN,OPEN,NaN,NaT,NaN,NaN,NaN,NaN,OAK GLEN PARK,Oakland,CA,NaT
2,1148022,2021-09-16 21:37:36,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.84456257751645, -122.26704955148797)",OPEN,NaN,NaT,6051460.077,2134803.248,CCD1,11X,5911 MACCALL ST,Oakland,CA,NaT
3,1148574,2021-09-20 13:30:43,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.841942007577984, -122.27307401524136)",CLOSED,NaN,2021-12-01 14:06:40,6049702.742,2133882.102,CCD1,10Y,878 57TH ST,Oakland,CA,72 days 00:35:57
4,1149273,2021-09-22 13:20:59,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.8424970155393, -122.2694559662867)",OPEN,NaN,NaT,6050751.127,2134064.386,CCD1,11X,5714 MARTIN LUTHER KING JR WAY,Oakland,CA,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862926,1164947,2021-11-17 08:29:12,Phone,Traffic - Repair Pavement Striping,TRAFFIC,"(37.74587673008652, -122.15610217415544)",WOCREATE,NaN,NaT,6082854.999,2098289.251,CCD7,32Y,100TH AV & MACARTHUR BLVD,Oakland,CA,NaT
862927,1165400,2021-11-18 15:04:00,Email,City Bldg - Electrical Inside/Outside,BLDGMAINT,"(37.80754717953511, -122.27516788700873)",CLOSED,NaN,2021-11-23 14:21:12,6048860.675,2121371.814,CCD3,04X,1601 MARTIN LUTHER KING JR WAY,Oakland,CA,4 days 23:17:12
862928,1167199,2021-11-29 09:04:08,Email,"Spillage - Trash, Recycle, Compost Spillage",RECYCLING,"(37.794757314318225, -122.19210995683491)",OPEN,NaN,NaT,6072767.360,2116271.016,CCD4,25X,4063 PATTERSON AVE,Oakland,CA,NaT
862929,1164502,2021-11-15 16:52:09,SeeClickFix,"Illegal Dumping - debris, appliances, etc.",ILLDUMP,"(37.76940868428093, -122.18206644059013)",CLOSED,NaN,2021-11-27 15:26:44,6075503.020,2106990.208,CCD6,30X,6643 BANCROFT AVE,Oakland,CA,11 days 22:34:35


## Explore data

In order to come up with a question for this data, we might have to explore it a bit to see if there are important details in the data that have been overlooked.

### Categories of incidents in 2019

I'm curious about the different categories of incidents in the year 2019.

First, I'll create a new dataframe `oakland_311_2019` that limits the `oakland_311` data to just the cases that were open in 2019.

In [8]:
oakland_311_2019 = oakland_311[
    (oakland_311['DATETIMEINIT'] >= '2019-01-01') &
    (oakland_311['DATETIMEINIT'] < '2020-01-01')
].copy()

In [9]:
oakland_311_2019

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT,PROBADDRESS,City,State,Close_Time
410,924852,2019-07-15 15:06:32,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.758713304033854, -122.18054475905758)",OPEN,NaN,NaT,6075872.851,2103088.527,CCD6,30X,7845 ASH ST,Oakland,CA,NaT
5215,863362,2019-01-03 09:59:00,Other,Parking - Abandoned Vehicle,POLICE,NaN,CLOSED,NaN,2019-01-03 10:24:08,NaN,NaN,NaN,NaN,5224 E 1OTH ST,Oakland,CA,0 days 00:25:08
5216,863684,2019-01-04 10:08:16,Phone,Litter in Parks,PARKS,"(37.84754948342958, -122.25754314069401)",CLOSED,NaN,2019-01-25 14:42:46,6054224.998,2135838.979,CCD1,12Y,COLBY PARK,Oakland,CA,21 days 04:34:30
5217,864060,2019-01-06 11:26:51,SeeClickFix,Homeless Encampment,HE_CLEAN,"(37.79677419976973, -122.27360019943853)",CANCEL,NaN,NaT,6049239.153,2117441.242,CCD3,01X,381 5TH ST,Oakland,CA,NaT
5218,864810,2019-01-08 14:31:54,Email,PG&E,OTHER,"(37.841298129045896, -122.2538281790993)",REFERRED,"PG&E,",NaT,6055254.808,2133542.976,CCD1,12Y,432 HUDSON ST,Oakland,CA,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688515,973481,2019-12-30 23:44:56,SeeClickFix,Parking - Enforcement,PARKING,"(37.76658332984048, -122.16632782739566)",CLOSED,NaN,2019-12-31 08:43:37,6080033.114,2105880.223,CCD6,30Y,7862 NEY AVE,Oakland,CA,0 days 08:58:41
688536,973391,2019-12-30 14:58:36,SeeClickFix,City Bldg - Electrical Inside/Outside,BLDGMAINT,"(37.800950001002086, -122.26367999714363)",CLOSED,NaN,2020-01-30 13:39:20,6052133.605,2118907.310,CCD2,03X,OAKLAND PUBLIC LIBRARY-MAIN BRANCH,Oakland,CA,30 days 22:40:44
688554,973735,2019-12-31 15:51:24,SeeClickFix,"Illegal Dumping - debris, appliances, etc.",ILLDUMP,"(37.796790482331915, -122.25177508141975)",CLOSED,NaN,2020-01-02 19:12:46,6055544.223,2117328.401,CCD2,19X,531 FOOTHILL BLVD,Oakland,CA,2 days 03:21:22
688555,973259,2019-12-30 11:14:01,SeeClickFix,"Illegal Dumping - debris, appliances, etc.",ILLDUMP,"(37.8393042922947, -122.27636840220823)",CLOSED,NaN,2020-01-28 13:28:08,6048733.393,2132939.854,CCD1,10X,950 55TH ST,Oakland,CA,29 days 02:14:07


One thing I'm seeing immediately is that the index of this new dataframe `oakland_311_2019` looks very disorganized. It's no longer sequential, many of the dates are in the wrong order. I can reset the index to make it sequential by using `df.reset_index(drop=True)`. (We learned this last lecture.)

```python
berkeley_311_2019 = berkeley_311_2019.reset_index(drop=True)
```

Alternatively, instead of copying the original dataframe with df.copy(), we can reset the index at the same time we subset the data:

In [10]:
oakland_311_2019 = oakland_311[
    (oakland_311['DATETIMEINIT'] >= '2019-01-01') &
    (oakland_311['DATETIMEINIT'] < '2020-01-01')
].reset_index(drop=True)

In [11]:
oakland_311_2019

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT,PROBADDRESS,City,State,Close_Time
0,924852,2019-07-15 15:06:32,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.758713304033854, -122.18054475905758)",OPEN,NaN,NaT,6075872.851,2103088.527,CCD6,30X,7845 ASH ST,Oakland,CA,NaT
1,863362,2019-01-03 09:59:00,Other,Parking - Abandoned Vehicle,POLICE,NaN,CLOSED,NaN,2019-01-03 10:24:08,NaN,NaN,NaN,NaN,5224 E 1OTH ST,Oakland,CA,0 days 00:25:08
2,863684,2019-01-04 10:08:16,Phone,Litter in Parks,PARKS,"(37.84754948342958, -122.25754314069401)",CLOSED,NaN,2019-01-25 14:42:46,6054224.998,2135838.979,CCD1,12Y,COLBY PARK,Oakland,CA,21 days 04:34:30
3,864060,2019-01-06 11:26:51,SeeClickFix,Homeless Encampment,HE_CLEAN,"(37.79677419976973, -122.27360019943853)",CANCEL,NaN,NaT,6049239.153,2117441.242,CCD3,01X,381 5TH ST,Oakland,CA,NaT
4,864810,2019-01-08 14:31:54,Email,PG&E,OTHER,"(37.841298129045896, -122.2538281790993)",REFERRED,"PG&E,",NaT,6055254.808,2133542.976,CCD1,12Y,432 HUDSON ST,Oakland,CA,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110923,973481,2019-12-30 23:44:56,SeeClickFix,Parking - Enforcement,PARKING,"(37.76658332984048, -122.16632782739566)",CLOSED,NaN,2019-12-31 08:43:37,6080033.114,2105880.223,CCD6,30Y,7862 NEY AVE,Oakland,CA,0 days 08:58:41
110924,973391,2019-12-30 14:58:36,SeeClickFix,City Bldg - Electrical Inside/Outside,BLDGMAINT,"(37.800950001002086, -122.26367999714363)",CLOSED,NaN,2020-01-30 13:39:20,6052133.605,2118907.310,CCD2,03X,OAKLAND PUBLIC LIBRARY-MAIN BRANCH,Oakland,CA,30 days 22:40:44
110925,973735,2019-12-31 15:51:24,SeeClickFix,"Illegal Dumping - debris, appliances, etc.",ILLDUMP,"(37.796790482331915, -122.25177508141975)",CLOSED,NaN,2020-01-02 19:12:46,6055544.223,2117328.401,CCD2,19X,531 FOOTHILL BLVD,Oakland,CA,2 days 03:21:22
110926,973259,2019-12-30 11:14:01,SeeClickFix,"Illegal Dumping - debris, appliances, etc.",ILLDUMP,"(37.8393042922947, -122.27636840220823)",CLOSED,NaN,2020-01-28 13:28:08,6048733.393,2132939.854,CCD1,10X,950 55TH ST,Oakland,CA,29 days 02:14:07


#### Let's see all the unique values of **REQCATEGORY**

You can call `series.unique()` on a column:

In [12]:
oakland_311_2019['REQCATEGORY'].unique()

array(['POLICE', 'PARKS', 'HE_CLEAN', 'OTHER', 'RECYCLING', 'TREES',
       'ILLDUMP', 'ELECTRICAL', 'TRAFFIC', 'PARKING', 'METER_REPAIR',
       'BLDGMAINT', 'ROW_INSPECTORS', 'ROW_STREETSW', 'FIRE', 'KOCB',
       'DRAINAGE', 'SEWERS', 'GRAFFITI', 'STREETSW', 'TRAFFIC_ENGIN',
       'CUT_CLEAN', 'CW_DIT_GIS', 'WATERSHED', nan], dtype=object)

Some of the category names are not that easy to identify and seem much more consolidated than the Berkeley 311 data.

I'm interested in getting a count of categories for 2019. How can I achieve this?

#### Aggregate with `df.groupby()`

To aggregate the data, we're going to use a method called `df.groupby()`. Normally, when we group data, we'll group them by columns, like so:

```python
df.groupby(['Column 1', 'Column 2'])
```

You can also just group by a single column, like we're doing below:

In [13]:
oakland_311_2019.groupby(['REQCATEGORY'])

Running a `df.groupby()` doesn't do anything on its own, it just creates a pandas DataFrameGroupBy object. You have to follow it up with some kind of other method. Below, we're calling `df.count()` on the DataFrameGroupBy object.

In [14]:
oakland_311_2019.groupby(['REQCATEGORY']).count()

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT,PROBADDRESS,City,State,Close_Time
REQCATEGORY,,,,,,,,,,,,,,,,
BLDGMAINT,7638,7638,7638,7638,7584,7638,15,6829,7584,7584,7318,7336,7638,7638,7638,6829
CUT_CLEAN,529,529,529,529,529,529,20,424,529,529,502,503,529,529,529,424
CW_DIT_GIS,28,28,28,28,3,28,0,27,3,3,3,3,28,28,28,27
DRAINAGE,1409,1409,1409,1409,1386,1409,14,1354,1386,1386,1375,1383,1409,1409,1409,1354
ELECTRICAL,2504,2504,2504,2504,2488,2504,1,2236,2488,2488,2370,2378,2504,2504,2504,2236
FIRE,574,574,574,574,571,574,4,100,571,571,568,571,574,574,574,100
GRAFFITI,3918,3918,3918,3918,3899,3918,113,3626,3899,3899,3690,3701,3918,3918,3918,3626
HE_CLEAN,3281,3281,3281,3281,3239,3281,15,150,3239,3239,3220,3225,3281,3281,3281,150
ILLDUMP,33826,33826,33826,33826,33631,33826,72,28914,33631,33631,31485,31671,33817,33826,33826,28914


OK! So that's a new dataframe, with a little too much info. The "counts" are also different across columns. For "count" to make sense, you need to limit the dataframe to a column with zero null cells. Since we know **REQUESTID** has unique values, we just need that one. 

Let's also set the results of the groupby/count to a new dataframe called `category_counts_2019`. You won't need to use `df.copy()` here because pandas will create a new dataframe instance with `df.groupby().count()`.

In [15]:
category_counts_2019 = oakland_311_2019.groupby(['REQCATEGORY']).count()

Now let's subset just the one column, **REQUESTID**, from `category_counts_2019`:

In [16]:
category_counts_2019 = category_counts_2019[['REQUESTID']].copy()
category_counts_2019

,REQUESTID
REQCATEGORY,
BLDGMAINT,7638
CUT_CLEAN,529
CW_DIT_GIS,28
DRAINAGE,1409
ELECTRICAL,2504
FIRE,574
GRAFFITI,3918
HE_CLEAN,3281
ILLDUMP,33826


Let's look at that code one more time.
```python
category_counts_2019 = category_counts_2019[['REQUESTID']].copy()
```

The outer set of brackets is used to select the data. Within the outer set of brackets is `['REQUESTID']`. That is a list of the columns. If we wanted to select two columns, we might have written:

```python
category_counts_2019 = category_counts_2019[['REQUESTID', 'DATETIMEINIT']].copy()
```

#### Resetting the index

In this dataframe, the index is no longer a series of sequential integers like we've seen before. We want **REQCATEGORY** to be a column, not an index, because that makes the dataframe easier to use.

We're going to use `df.reset_index()`. This time, we're not going to use the `drop=True` argument because we want to create a wholly new index.

In [17]:
category_counts_2019 = category_counts_2019.reset_index()

In [18]:
category_counts_2019

,REQCATEGORY,REQUESTID
0,BLDGMAINT,7638
1,CUT_CLEAN,529
2,CW_DIT_GIS,28
3,DRAINAGE,1409
4,ELECTRICAL,2504
5,FIRE,574
6,GRAFFITI,3918
7,HE_CLEAN,3281
8,ILLDUMP,33826
9,KOCB,839


#### Rename columns

Let's change the column names, while we're at it. 
I want to change the name of the **REQUESTID** column to represent an actual count, like **Case_Count**.

You can replace _all_ the columns in a dataframe with the following code:

```python
category_counts_2019.columns = ['REQCATEGORY', 'Case_Count']
```

If you have a lot of columns, that's going to be a long list. But if you have a lot of columns to rename, the method above might be easier. If you have only one column to rename out of many columns, you'll want to use the following code:

```python
category_counts_2019.rename(columns={'REQUESTID': 'Case_Count'}, inplace=True)
```

The first argument in the `df.rename()` method is `columns`. And what do we set columns to? We set it to a Python dictionary where the "key" is the original column name and the "value" is the new column name: `{'Case_ID': 'Case_Count'}`. 

The second argument is `inplace=True`. That tells us to change the `category_counts_2019` "in place" or without having to reset the dataframe variable. A lot of the methods in pandas return a new dataframe instead of altering the original dataframe. An alternative to using `inplace` is the following code:

```python
category_counts_2019 = category_counts_2019.rename(columns={'REQUESTID': 'Count'})
```

In [19]:
category_counts_2019.rename(columns={'REQUESTID': 'Case_Count'}, inplace=True)

Let's take a look at our nicely named summary table:

In [20]:
category_counts_2019

,REQCATEGORY,Case_Count
0,BLDGMAINT,7638
1,CUT_CLEAN,529
2,CW_DIT_GIS,28
3,DRAINAGE,1409
4,ELECTRICAL,2504
5,FIRE,574
6,GRAFFITI,3918
7,HE_CLEAN,3281
8,ILLDUMP,33826
9,KOCB,839


Now we see see the 'REQCATEGORY' nicely organized in numerical order

Now, I'll sort the dataframe by **Case_Count**:

In [21]:
category_counts_2019 = category_counts_2019.sort_values(by=['Case_Count'], ascending=False).copy()
category_counts_2019

,REQCATEGORY,Case_Count
8,ILLDUMP,33826
14,POLICE,22774
11,OTHER,8016
0,BLDGMAINT,7638
19,STREETSW,7064
6,GRAFFITI,3918
7,HE_CLEAN,3281
12,PARKING,3144
22,TREES,2761
4,ELECTRICAL,2504


It looks like Illegal Dumping and Police cases were among the top issues in 2019. Might be worth looking into some of the sub-categories later.

At this point, you can see the index is out of order now. You can use `df.reset_index(drop=True)` if you like. You could have also replaced `df.copy()` in the previous cell with `df.reset_index(drop=True)`.

In [22]:
category_counts_2019 = category_counts_2019.reset_index(drop=True)
category_counts_2019

,REQCATEGORY,Case_Count
0,ILLDUMP,33826
1,POLICE,22774
2,OTHER,8016
3,BLDGMAINT,7638
4,STREETSW,7064
5,GRAFFITI,3918
6,HE_CLEAN,3281
7,PARKING,3144
8,TREES,2761
9,ELECTRICAL,2504


Now we can see the  of each 'REQCATEGORY' in descending order. Some of these categories are a little hard to decipher expecially the 'REQCATEGORY' 'CW_DIT_GS'. I wonder what that means and if it is relevant to our reporting. Also 'OTHER' is very broad as well.

#### Let's visualize this summary table!

Before we run the Altair code below, let's take a closer look:

```python
alt.Chart(category_counts_2019).mark_bar().encode(
    x='Case_Count',
    y='REQCATEGORY'
).properties(
    title='Oakland 311 cases in 2019'
)
```
The first part of the code `alt.Chart()` requires you to fill the first argument with a dataframe, in this case `category_counts_2019`.

The next part of the code `mark_bar()` specifies a bar chart. (If you want a line chart, you'd use `mark_line()`.)

After that, `.encode()` tells Altair which columns to use for the `x` and `y` axes.

If you want to add a title, you'd use Altair's `.properties()` method.

In [23]:
alt.Chart(category_counts_2019).mark_bar().encode(
    x='Case_Count',
    y='REQCATEGORY'
).properties(
    title='Oakland 311 cases in 2019'
)

alt.Chart(...)

This is a really cool visualization that helps the audience compare the most to the least `REQCATEGORY's in 2019.

Annoyingly, this doesn't sort the chart in descending order, which I prefer. This is the code to do, it's a little more complicated:

```python
alt.Chart(category_counts_2019).mark_bar().encode(
    x='Case_Count',
    y=alt.Y('REQCATEGORY', sort='-x')
).properties(
    title='Oakland 311 cases in 2019'
)
```

Basically, you have to create a custom Y encoding with the format: `alt.Y('column_name', sort='-x')`. `-x` means the inverse of the x-axis, in this case. This is not intuitive, I think — it's just something you'd have to look up in the documentation.

In [24]:
alt.Chart(category_counts_2019).mark_bar().encode(
    x='Case_Count',
    y=alt.Y('REQCATEGORY', sort='-x')
).properties(
    title='Oakland 311 cases in 2019'
)

alt.Chart(...)

### Count how many incidents per year

The next thing I'd like to do is get a count of all the incidents by year. However, I know from the last notebook that the data for 2010 and 2021 are not complete. So I need to subset.

Below, I'm creating a new dataframe called `oakland_311_complete` that limits the `oakland_311` dataframe to ones in which the **Date_Opened** value starts on or after January 1, 2011 and is before January 1, 2021. 

In [25]:
oakland_311_complete = oakland_311[
    (oakland_311['DATETIMEINIT'] >= '2011-01-01') &
    (oakland_311['DATETIMEINIT'] < '2021-01-01')
].reset_index(drop=True)

#### Use df.groupby() with datetimes

It's a little tricky to group by datetimes. Instead of grouping by just a column name, we're going to have to use a method called `pd.Grouper`. 

Before we run the code below, let's look at the different arguments within the method:

```python
pd.Grouper(key='DATETIMEINIT', axis=0, freq='A')
```

The `key` argument lists the column. The `axis` argument is `0`. In pandas, axis 0 is rows and axis 1 means columns. That means you can do column-wise calculations if your data is shaped differently. 

The `freq` argument is `A`, which stands for "annual" or year (`Y` also works, but isn't documented). You can see other [frequency arguments](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases) in the official pandas documentation.

In [26]:
oakland_311_complete.groupby([pd.Grouper(key='DATETIMEINIT', axis=0, freq='A')])

Remember that running a `df.groupby()` doesn't do anything on its own; you have to chain that command with some kind of other method. Below, we're calling `df.count()` on the DataFrameGroupBy object. Finally, we're calling our new dataframe `annual_cases`.

In [27]:
annual_cases = oakland_311_complete.groupby([pd.Grouper(key='DATETIMEINIT', axis=0, freq='A')]).count()

In [28]:
annual_cases

,REQUESTID,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT,PROBADDRESS,City,State,Close_Time
DATETIMEINIT,,,,,,,,,,,,,,,,
2011-12-31,37995,37995,37995,37988,36988,37995,414,33973,36988,36988,34962,35021,37920,37995,37995,33973
2012-12-31,47294,47294,47294,47287,46233,47294,4776,36719,46233,46233,43922,44103,47220,47294,47294,36719
2013-12-31,56888,56888,56888,56884,55004,56888,6632,40377,55004,55004,51317,51416,56871,56888,56888,40377
2014-12-31,61826,61826,61826,61826,60769,61826,7395,45526,60769,60769,59102,59267,61799,61826,61826,45526
2015-12-31,66889,66889,66889,66869,65588,66889,7526,49276,65588,65588,64097,64316,66867,66889,66889,49276
2016-12-31,75932,75932,75932,75926,74372,75932,7681,54222,74372,74372,72466,72637,75908,75932,75932,54222
2017-12-31,80740,80740,80740,80740,79446,80740,8548,53068,79446,79446,76747,76967,80729,80740,80740,53068
2018-12-31,77851,77851,77851,77843,77066,77851,10313,50193,77066,77066,76193,76417,77829,77851,77851,50193
2019-12-31,110928,110928,110928,110911,109763,110928,6752,77582,109763,109763,104333,104765,110899,110928,110928,77582


I noticed during the pandemic years (2020) and also in 2019, there is a big gap between cases that have been initiated and cases that have been closed, obviously the pandemic plays a part in this.

Now let's subset just the one column, **REQUESTID**, from annual cases, then reset the index so that `DATETIMEINIT` becomes a new column:

In [29]:
annual_cases = annual_cases[['REQUESTID']].reset_index()
annual_cases

,DATETIMEINIT,REQUESTID
0,2011-12-31,37995
1,2012-12-31,47294
2,2013-12-31,56888
3,2014-12-31,61826
4,2015-12-31,66889
5,2016-12-31,75932
6,2017-12-31,80740
7,2018-12-31,77851
8,2019-12-31,110928
9,2020-12-31,103837


#### Rename columns

In [30]:
annual_cases.rename(columns={'REQUESTID': 'Case_Count'}, inplace=True)

Let's take a look at our nicely named summary table:

In [31]:
annual_cases

,DATETIMEINIT,Case_Count
0,2011-12-31,37995
1,2012-12-31,47294
2,2013-12-31,56888
3,2014-12-31,61826
4,2015-12-31,66889
5,2016-12-31,75932
6,2017-12-31,80740
7,2018-12-31,77851
8,2019-12-31,110928
9,2020-12-31,103837


Again, the 'Case_Count' seems much higher in 2019 and 2020

Let's create a new column in `annual_cases` called **Year**.

In [32]:
annual_cases['Year'] = annual_cases['DATETIMEINIT'].dt.year

In [33]:
annual_cases

,DATETIMEINIT,Case_Count,Year
0,2011-12-31,37995,2011
1,2012-12-31,47294,2012
2,2013-12-31,56888,2013
3,2014-12-31,61826,2014
4,2015-12-31,66889,2015
5,2016-12-31,75932,2016
6,2017-12-31,80740,2017
7,2018-12-31,77851,2018
8,2019-12-31,110928,2019
9,2020-12-31,103837,2020


At this point, I don't need the **DATETIMEINIT** column anymore. So I can subset the dataframe with just the two columns I need. 

In [34]:
annual_cases = annual_cases[['Year', 'Case_Count']].copy()

In [35]:
annual_cases

,Year,Case_Count
0,2011,37995
1,2012,47294
2,2013,56888
3,2014,61826
4,2015,66889
5,2016,75932
6,2017,80740
7,2018,77851
8,2019,110928
9,2020,103837


#### Visualize

In [36]:
alt.Chart(annual_cases).mark_bar().encode(
    x='Year',
    y='Case_Count'
)

alt.Chart(...)

The chart looks really cool but **Year** need to be fixed, we don't read '2020' as '2,020'

Let's make a very small alteration to the code.

Before you run the code below, notice that after `Year` there's a colon and an `O`. The `O` is shorthand for "ordinal," and tells Altair to treat `Year` as if it's a discrete quantity (a.k.a. integers), not a continuous quantity (e.g. a number with decimals). 

In [37]:
alt.Chart(annual_cases).mark_bar().encode(
    x='Year:O',
    y='Case_Count'
).properties(
    title='Oakland 311 calls: Number of cases'
)

alt.Chart(...)

The chart now looks more condensed and the **Year** is now correctly formatted. I just realized that there were more cases in 2019 than in 2020 and I wonder what the reason for this is? Obviously precautions became greater during 2020, so I'm worried if there cases that needed to be made but weren't because of the pandemic.

You can read about more [Altair encoding types](https://altair-viz.github.io/user_guide/encoding.html#encoding-data-types) in the documentation. It's helpful to get familiar with those encoding types in the event your chart doesn't look quite right. Try adjusting the encoding types on your own to see what happens.

### Median Close_Time by year

Now I'd like to try to get the median length of time it takes to close a case by year. I'm going to try something I think will work...

In [38]:
median_close_time = oakland_311_complete.groupby([pd.Grouper(key='DATETIMEINIT', axis=0, freq='A') ]).median()

In [39]:
median_close_time

,SRX,SRY
DATETIMEINIT,,
2011-12-31,6.060215e+06,2117932.000
2012-12-31,6.059595e+06,2118648.000
2013-12-31,6.058980e+06,2118542.710
2014-12-31,6.056907e+06,2119122.250
2015-12-31,6.056996e+06,2118843.890
2016-12-31,6.060108e+06,2117722.440
2017-12-31,6.061456e+06,2117412.630
2018-12-31,6.060343e+06,2117891.508
2019-12-31,6.061430e+06,2117555.865


It looks like that didn't work! Sometimes pandas doesn't work the way you want it to. Getting the median of a `timedelta` dtype is a feature that hasn't yet been built.

I'm instead going to create a new column called **Close_Time_Seconds** that converts **Close_Time** to seconds. Then I can calculate the median number of seconds.

In [40]:
oakland_311_complete['Close_Time_Seconds'] = oakland_311_complete['Close_Time'].dt.total_seconds()

Let's make sure the new column looks OK:

In [41]:
oakland_311_complete.head()

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT,PROBADDRESS,City,State,Close_Time,Close_Time_Seconds
0,978974,2020-01-18 16:19:49,SeeClickFix,Streets/Sidewalks - Other/Complex,BLDGMAINT,"(37.846975038495714, -122.28597679425833)",CLOSED,NaN,2021-12-02 12:37:27,6046012.569,2135785.268,CCD1,10X,OCEAN AVE & MARSHALL ST,Oakland,CA,683 days 20:17:38,59084258.0
1,369975,2011-05-12 14:43:40,Phone or Email,Sidewalk - Damage,ROW_STREETSW,"(37.796927666729566, -122.1983845122619)",OPEN,NaN,NaT,6070969.000,2117094.000,CCD4,22Y,3908 NORTON AV,Oakland,CA,NaT,NaN
2,1030386,2020-07-25 12:15:33,SeeClickFix,Sidewalk - Damage,STREETSW,"(37.8233750317369, -122.2536110578072)",OPEN,NaN,NaT,6055195.211,2127016.697,CCD1,09X,82 RIO VISTA AVE,Oakland,CA,NaT,NaN
3,924852,2019-07-15 15:06:32,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.758713304033854, -122.18054475905758)",OPEN,NaN,NaT,6075872.851,2103088.527,CCD6,30X,7845 ASH ST,Oakland,CA,NaT,NaN
4,1071498,2020-12-04 11:05:41,SeeClickFix,Parking - Abandoned Vehicle,POLICE,"(37.73195845242447, -122.17670059780059)",CLOSED,NaN,2021-12-04 16:45:21,6076809.430,2093328.199,CCD7,31Z,10770 EDES AVE,Oakland,CA,365 days 05:39:40,31556380.0


Let's run the groupby/median code we used before. Hopefully pandas will calculate **Close_Time_Seconds** this time around:

In [43]:
median_close_time = oakland_311_complete.groupby([pd.Grouper(key='DATETIMEINIT', axis=0, freq='A'), ]).median()

In [44]:
median_close_time

,SRX,SRY,Close_Time_Seconds
DATETIMEINIT,,,
2011-12-31,6.060215e+06,2117932.000,345590.0
2012-12-31,6.059595e+06,2118648.000,440865.0
2013-12-31,6.058980e+06,2118542.710,520178.0
2014-12-31,6.056907e+06,2119122.250,429549.5
2015-12-31,6.056996e+06,2118843.890,448532.0
2016-12-31,6.060108e+06,2117722.440,520235.5
2017-12-31,6.061456e+06,2117412.630,359351.0
2018-12-31,6.060343e+06,2117891.508,354400.0
2019-12-31,6.061430e+06,2117555.865,517240.0


We can now see the time length in seconds. It looks like (2013, 2016) took the longest turnaround time, I wonder why that is?

In [45]:
median_close_time = median_close_time[['Close_Time_Seconds']].reset_index()
median_close_time

,DATETIMEINIT,Close_Time_Seconds
0,2011-12-31,345590.0
1,2012-12-31,440865.0
2,2013-12-31,520178.0
3,2014-12-31,429549.5
4,2015-12-31,448532.0
5,2016-12-31,520235.5
6,2017-12-31,359351.0
7,2018-12-31,354400.0
8,2019-12-31,517240.0
9,2020-12-31,390935.0


Below, I'm creating a new column called **Year**, as we did before. I'm also going to convert seconds to days:

In [46]:
median_close_time['Year'] = median_close_time['DATETIMEINIT'].dt.year
median_close_time['Close_Time_Days'] = median_close_time['Close_Time_Seconds'] / 60 / 60 / 24

Let's take another look at our dataframe before we subset:

In [47]:
median_close_time

,DATETIMEINIT,Close_Time_Seconds,Year,Close_Time_Days
0,2011-12-31,345590.0,2011,3.999884
1,2012-12-31,440865.0,2012,5.102604
2,2013-12-31,520178.0,2013,6.020579
3,2014-12-31,429549.5,2014,4.971638
4,2015-12-31,448532.0,2015,5.191343
5,2016-12-31,520235.5,2016,6.021244
6,2017-12-31,359351.0,2017,4.159155
7,2018-12-31,354400.0,2018,4.101852
8,2019-12-31,517240.0,2019,5.986574
9,2020-12-31,390935.0,2020,4.524711


In [48]:
median_close_time = median_close_time[['Year', 'Close_Time_Days']].copy()

In [49]:
median_close_time

,Year,Close_Time_Days
0,2011,3.999884
1,2012,5.102604
2,2013,6.020579
3,2014,4.971638
4,2015,5.191343
5,2016,6.021244
6,2017,4.159155
7,2018,4.101852
8,2019,5.986574
9,2020,4.524711


It's really cool to visualize the median amount of time it took to close a case in days. I'm still interested in what happened in 2013, 2016 and also 2019.

In [50]:
alt.Chart(median_close_time[['Year', 'Close_Time_Days']]).mark_bar().encode(
    x='Year:O',
    y='Close_Time_Days',
).properties(
    title='Oakland 311 calls: Median resolution time'
)

alt.Chart(...)

Now, we can clearly see the difference in median resolution time, with 2013, 2016 and 2019 with the longest interval to `Close_Time_Days`

### Merge two dataframes

Now I'd like to merge `median_close_time` and `annual_cases`. Why? Mostly because I'd like to teach you how to merge dataframes. But you can get a neat summary table this way. Let's look at both dataframes again:

In [51]:
annual_cases

,Year,Case_Count
0,2011,37995
1,2012,47294
2,2013,56888
3,2014,61826
4,2015,66889
5,2016,75932
6,2017,80740
7,2018,77851
8,2019,110928
9,2020,103837


In [52]:
median_close_time

,Year,Close_Time_Days
0,2011,3.999884
1,2012,5.102604
2,2013,6.020579
3,2014,4.971638
4,2015,5.191343
5,2016,6.021244
6,2017,4.159155
7,2018,4.101852
8,2019,5.986574
9,2020,4.524711


Let's look at the arguments in `pd.merge()` before we run it:

```python
pd.merge(
    df1,
    df2,
    how='outer', # other options: 'inner', 'left', 'right'
    on='Year',
    validate='1:1' # options: '1:m', 'm:m', 'm:1'
)
```
1. The first argument is the left-hand dataframe. The second argument is the right-hand dataframe. Why is it important that there's an order to dataframes? 

2. The `how` argument tells pandas how we'll merge the two dataframes. In this case, we'll use `outer`. But we could also use `left`, `right`, or `inner`. What does this mean? Let's take a look at the lecture slides to view a visual. In this case, it doesn't matter which value we use for `how` because both dataframes have 10 rows with matching years.

3. The `on` argument tells pandas which column key we're going to match on. In this case, we want the years to match up.

4. The `validate` argument is optional, but I recommend you learn how to use it. The value we used, `'1:1'` means that 1 row in the left-hand dataframe will match up to exactly 1 row in the right-hand dataframe. The option `1:m` means that 1 row in the left-hand dataframe could match up to **many** rows in the right-hand dataframe. (Any time you use `m`, you're telling pandas that there _might_ be multiple matches.)

In [53]:
annual_summary = pd.merge(
    annual_cases,
    median_close_time,
    on='Year',
    how='outer',
    validate='1:1'
)

In [54]:
annual_summary

,Year,Case_Count,Close_Time_Days
0,2011,37995,3.999884
1,2012,47294,5.102604
2,2013,56888,6.020579
3,2014,61826,4.971638
4,2015,66889,5.191343
5,2016,75932,6.021244
6,2017,80740,4.159155
7,2018,77851,4.101852
8,2019,110928,5.986574
9,2020,103837,4.524711


Now that I can see a full picture of the summary. I can see that 2019 had the highest `Case_Count` which helps us understand why `Close_Time_Days` took much longer. I am not sure why 2013 and 2016 took the longest interval amount of time but this is when we can look into the data and see what type of cases were reported and investigate why they took longer to close. It would also be helpful to understand what was happening at the city at the time to better understand why more calls were made.